## Test #2 - Apache Beam with Dataflow

First, we define the dependencies that we wish to load from the Maven repositories.  Since we are performing this outside of Jupyter:

```
<?xml version="1.0" encoding="UTF-8"?>

<project xmlns="http://maven.apache.org/POM/4.0.0"
  xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 http://maven.apache.org/xsd/maven-4.0.0.xsd">
  <modelVersion>4.0.0</modelVersion>

  <groupId>jupyter</groupId>
  <artifactId>jupyter</artifactId>
  <version>1</version>

  <dependencies>
    <dependency>
      <groupId>org.apache.beam</groupId>
      <artifactId>beam-sdks-java-core</artifactId>
      <version>2.40.0</version>
    </dependency>

    <dependency>
      <groupId>org.apache.beam</groupId>
      <artifactId>beam-runners-direct-java</artifactId>
      <version>2.40.0</version>
      <scope>runtime</scope>
    </dependency>

    <dependency>
      <groupId>org.apache.beam</groupId>
      <artifactId>beam-runners-google-cloud-dataflow-java</artifactId>
      <version>2.40.0</version>
    </dependency>

    <dependency>
      <groupId>org.slf4j</groupId>
      <artifactId>slf4j-jdk14</artifactId>
      <version>1.7.32</version>
    </dependency>
  </dependencies>
</project>
```

next we run:

```
mvn dependency:copy-dependencies
```

Finally, we set an environment variable and re-run Jupyter:

```
export IJAVA_CLASSPATH="/pathToDepenencies/*"
jupyter notebook
```

Next we define our core code that builds the Apache Beam pipeline:

In [1]:
import java.util.Arrays;
import java.util.List;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.PipelineResult;
import org.apache.beam.sdk.coders.StringUtf8Coder;
import org.apache.beam.sdk.options.Default;
import org.apache.beam.sdk.options.Description;
import org.apache.beam.sdk.options.PipelineOptionsFactory;
import org.apache.beam.sdk.options.StreamingOptions;
import org.apache.beam.sdk.transforms.Create;
import org.apache.beam.sdk.transforms.MapElements;
import org.apache.beam.sdk.values.PCollection;
import org.apache.beam.sdk.values.TypeDescriptors;
import org.apache.beam.sdk.transforms.SimpleFunction;

public interface Options extends StreamingOptions {
  @Description("Input text to print.")
  @Default.String("My input text")
  String getInputText();

  void setInputText(String value);
}

public class X {
public static PCollection<String> buildPipeline(Pipeline pipeline, String inputText) {
  return pipeline
    .apply("Create elements", Create.of(Arrays.asList("Hello!", "World!", inputText)))
    .apply("Print elements",
      MapElements.via(
          new SimpleFunction<String, String>() {
              @Override
              public String apply(String text) {
                  System.out.println(text);
                  return text;
              }
          }
    )
  );
}
}

Finally, we run the pipeline and see the output:

In [4]:
public class Y {
    public static void main() {
        String args[] = new String[] {"--runner=DataflowRunner","--region=us-central1","--inputText=XXX YYY"};

        Options options = PipelineOptionsFactory.fromArgs(args).withValidation().as(Options.class);
        Pipeline pipeline = Pipeline.create(options);
        X.buildPipeline(pipeline, options.getInputText());
        PipelineResult result = pipeline.run();
    }
}


In [3]:
System.out.println(result.getState());

RUNNING


In [5]:
Y.main();